In [1]:
!jupyter notebook list

Currently running servers:
http://localhost:8888/?token=bdf0dcd7590a6d969426b87c5834d95ee22764493d04beb3 :: /Users/jooyeonjamielee
http://localhost:8889/?token=fdf945a99fba2fa97b1b38af29cd3914513fe4537102abc0 :: /Users/jooyeonjamielee


In [3]:
from gensim.models import LdaModel, HdpModel
from gensim.corpora.dictionary import Dictionary
from gensim.models.coherencemodel import CoherenceModel
import gensim
import nltk
from nltk.corpus import stopwords
import pyLDAvis.gensim
import os
import sys
import matplotlib.pyplot as plt

reload(sys).setdefaultencoding("utf8")

mallet_path = '/Users/jooyeonjamielee/Documents/external/mallet-2.0.8/bin/mallet'

def get_corpus_dictionary():
    """Crafts a toy corpus and the dictionary associated."""
    # Toy corpus.
    # corpus = [
    #     ['carrot', 'salad', 'tomato'],
    #     ['carrot', 'salad', 'dish'],
    #     ['tomato', 'dish'],
    #     ['tomato', 'salad'],
    # 
    #     ['car', 'break', 'highway'],
    #     ['highway', 'accident', 'car'],
    #     ['moto', 'break'],
    #     ['accident', 'moto', 'car']
    # ]
    
    corpus = []
    with open('reviews_pmc_4443_0.7.txt', 'r') as f:
        contents = f.readlines()
        stopWords = set(stopwords.words('english'))
    
        for line in contents:
            sentence = nltk.word_tokenize(line)
            words = []
            for w in sentence:
                if w not in stopWords \
                and not w.isdigit() \
                and w not in '<' \
                and w not in '>'\
                and w not in '('\
                and w not in ')'\
                and w not in ':'\
                and w not in ']'\
                and w not in '['\
                and w not in '='\
                and w not in ','\
                and w not in ';'\
                and w != 'eos':
#                     if w == 'eos':
#                         w = '<eos>'
                    words.append(w)
            corpus.append(words)
    print corpus[:10]
    dictionary = Dictionary(corpus)
    lemmatized = corpus

    # Transforming corpus with dictionary.
    corpus = [dictionary.doc2bow(doc) for doc in corpus]
    print corpus[:10]
    # Building reverse index.
    for (token, uid) in dictionary.token2id.items():
        dictionary.id2token[uid] = token
    
    return corpus, dictionary, lemmatized

def lda():
    """Trains a LDA model and tests the html outputs."""
    corpus, dictionary, lemmatized = get_corpus_dictionary()

    lda = LdaModel(corpus=corpus,
                   num_topics=20)
    print 'lda: ', lda
    print dictionary
    data = pyLDAvis.gensim.prepare(lda, corpus, dictionary)
    pyLDAvis.display(data)
    print 'data'
    pyLDAvis.save_html(data, 'index_lda_20_0.7.html')
    print 'done'
    return dictionary,  corpus, lemmatized
    
def compute_coherence_values(dictionary, corpus, texts, limit, start=2, step=3):
    """
    Compute c_v coherence for various number of topics

    Parameters:
    ----------
    dictionary : Gensim dictionary
    corpus : Gensim corpus
    texts : List of input texts
    limit : Max num of topics

    Returns:
    -------
    model_list : List of LDA topic models
    coherence_values : Coherence values corresponding to the LDA model with respective number of topics
    """
    coherence_values = []
    model_list = []
    for num_topics in range(start, limit, step):
        model = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus, num_topics=num_topics, id2word=id2word)
        model_list.append(model)
        coherencemodel = CoherenceModel(model=model, texts=texts, dictionary=dictionary, coherence='c_v')
        coherence_values.append(coherencemodel.get_coherence())
    return model_list, coherence_values


if __name__ == "__main__":
    id2word, corpus, lemmatized = lda()
    # Can take a long time to run.
#     print id2word
    model_list, coherence_values = compute_coherence_values(dictionary=id2word, corpus=corpus, texts=lemmatized, start=2, limit=40, step=6)
    limit=40; start=2; step=6;
    x = range(start, limit, step)
    plt.plot(x, coherence_values)
    plt.xlabel("Num Topics")
    plt.ylabel("Coherence score")
    plt.legend(("coherence_values"), loc='best')
    plt.show()
    # test_hdp()



<Figure size 640x480 with 1 Axes>